In [1]:
import pandas as pd
import numpy as np

from mod.logs import Logger
from mod.dao import MyConn, SqlStatement
from mod.clean.pca import check_series_num, prep_pca, reduce_dim

from data_lake import PT_TABLE, OP_TABLE, DX_TABLE

In [2]:
# Prepare modules
l = Logger()
conn = MyConn("127.0.0.1", "hecon", l, False)
builder = SqlStatement()

In [5]:
# Load data from database
pt = conn.wrap(builder.read_data, table=PT_TABLE)
op = conn.wrap(builder.read_data, table=OP_TABLE)
dx = conn.wrap(builder.read_data, table=DX_TABLE)


        select * from hecon.pt_dl
        

        select * from hecon.op_dl
        

        select * from hecon.dx_dl
        


# 1. Regarding Duplication Removal?

In [10]:
# Remove duplicates?
# Let's write a simple duplicate removal code. 

def remove_duplicates(data: np.ndarray, pivot_order=None) -> pd.DataFrame:
    if pivot_order is None:
        pivot_order = {"index": 1, "columns": 0, "values": 2}

    # Create data
    df = pd.DataFrame(data)
    df_pivot = df.pivot(**pivot_order)

    # Identify duplicate rows, where the current row is the same as the previous row
    df_pivot["is_dup"] = df_pivot.duplicated(
        subset=df_pivot.columns.difference(["dates"]),
        keep="first",
    ) & ~df_pivot.duplicated(
        subset=df_pivot.columns.difference(["dates"]),
        keep="last",
    )

    # Keep only the rows that are not marked as duplicates
    df_cleaned = df_pivot[
        ~df_pivot["is_dup"] |
        df_pivot.duplicated(
            subset=df_pivot.columns.difference(["dates"]),
            keep="last"
        )
        ].drop(columns="is_dup")
    df_cleaned.reset_index(drop=True, inplace=True)
    return df_pivot


# PT Data
pt_original = pd.DataFrame(pt)
pt_original = pt_original.pivot(index=1, columns=0, values=2).reset_index()
pt_remove_dup = remove_duplicates(pt)
print(
    f"PT data: Before {pt_original.shape} / After Duplicate Removal {pt_remove_dup.shape}"
)

# OP Data
op_original = pd.DataFrame(op)
op_original = op_original.pivot(index=1, columns=0, values=2).reset_index()
op_remove_dup = remove_duplicates(op)
print(
    f"OP data: Before {op_original.shape} / After Duplicate Removal {op_remove_dup.shape}"
)

# DX Data
dx_original = pd.DataFrame(dx)
dx_original = dx_original.pivot(index=1, columns=0, values=2).reset_index()
dx_remove_dup = remove_duplicates(dx)
print(
    f"DX data: Before {dx_original.shape} / After Duplicate Removal {dx_remove_dup.shape}"
)


PT data: Before (243, 16) / After Duplicate Removal (243, 16)
OP data: Before (243, 43) / After Duplicate Removal (243, 43)
DX data: Before (243, 46) / After Duplicate Removal (243, 46)



```python
PT data: Before (243, 16) / After Duplicate Removal (243, 16)
OP data: Before (243, 43) / After Duplicate Removal (243, 43)
DX data: Before (243, 46) / After Duplicate Removal (243, 46)
```

<h3><i>No datapoints were removed</i></h3>

243개의 시계열 개수는 연구자가 주장한 243개의 총 시계열과 일치함. 

# 2. Regarding Zero Inflation ?
|	| 0개수 | 전체 개수 | 0 비율 | 연구자 자료 |
|---|------|--------|----|-----------|
|OP | 7095 | 10206 | 0.6952 | 약 70% |
|DX | 7467 | 10935 | 0.682853224 | 약 69% |

* 90% 넘는 물품만 제거한다고 하였기 때문에, 70% 인 물품 제거 안함. 
* 이미 시점 개수가 243개인 것을 보아, 이미 정리된 자료를 준 것으로 추정됨.

# 3. Regarding Principal Component Analysis

In [6]:
# Principal Component Analysis

# Reshape data by (TimeFrame * Series) Matrices using `pivot`
ptdf = pd.DataFrame(pt)
opdf = pd.DataFrame(op)
dxdf = pd.DataFrame(dx)

ptdf = ptdf.pivot(index=1, columns=0, values=2)
opdf = opdf.pivot(index=1, columns=0, values=2)
dxdf = dxdf.pivot(index=1, columns=0, values=2)

# Without date column
print("PT time series:", check_series_num(ptdf))
print("OP time series:", check_series_num(opdf))
print("DX time series:", check_series_num(dxdf))
print("\n")


PT time series: 15
OP time series: 42
DX time series: 45




* OP와 DX 데이터 프레임에서 시계열 개수는각각 42개, 45개로 파워포인트 5페이지의 "42개 변수", "45개 변수"와 일치.
* PT를 제외하고, OP + DX 데이터 프레임을 합한 (243 * 87) 매트릭스에서 PCA를 통해 차원 축소. 목표 (87개의 시계열 -> 19개의 시계열)

In [7]:
# Prep PCA Data. - Use only 215 data points to prevent data leakage
data = prep_pca([opdf, dxdf], 215)
print("OP & DX data merged by dates. Shape:", data.shape)

# Perform PCA with Standardization as default
result =reduce_dim([opdf, dxdf], True, 215)

OP & DX data merged by dates. Shape: (215, 87)
        PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0 -0.722300 -2.652508 -4.324250 -0.498422 -1.662924  2.286756  2.894187   
1 -0.307477 -2.272311 -4.924756  0.040354 -1.644941  2.674217  3.793473   
2 -0.307477 -2.272311 -4.924756  0.040354 -1.644941  2.674217  3.793473   

        PC8       PC9      PC10      PC11      PC12      PC13      PC14  \
0 -0.930957 -0.665034  2.497637  1.125673 -1.456916 -1.960308 -0.450932   
1 -0.784862 -0.892286  1.445341  0.995391 -1.059013 -2.050220  0.381942   
2 -0.784862 -0.892286  1.445341  0.995391 -1.059013 -2.050220  0.381942   

       PC15      PC16      PC17      PC18      PC19  
0  1.651021 -0.620240  0.412349  0.757394  1.199475  
1  0.938856  0.029288  0.729068  0.456795  0.860014  
2  0.938856  0.029288  0.729068  0.456795  0.860014  
Explained Variance by each component: [0.09258922 0.07034213 0.05868283 0.05481004 0.0493887  0.04445913
 0.04292404 0.04080997 0.03953

Result of the PCA shows that
* The total explained variance for selected 19 components are indeed account up to 75% of the total variance. 

# 4. Add datetime information